In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
import collections
import re
from tqdm import tqdm
import itertools

from itertools import islice, combinations
from collections import Counter, defaultdict

from itertools import chain
import ast


import statistics
import numpy as np

import math
import nltk
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures


In [ ]:
# ES = pd.read_csv('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/ES/2016_1115_G06F_ES_raw.csv')
# ES.to_pickle("C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/data/pickledDF/2016_1115_G06F_ES_raw.pkl")
# toEval = pd.read_csv('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/toEval/2016_G06F_patents_toEval.csv')
# toEval.to_pickle('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/data/pickledDF/2016_G06F_patents_toEval.pkl')

In [2]:
toEval = pd.read_pickle('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/pickledDF/2016_G06F_patents_toEval.pkl') 
ES = pd.read_pickle('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/pickledDF/2016_1115_G06F_ES_raw.pkl')
print("toEval lenght: ", len(toEval))
print("KS lenght: ", len(ES))

# Création différents jdD
toEval_ipc = list([ast.literal_eval(i) for i in toEval.sec_ipc])
ES_ipc = list(set(chain.from_iterable([ast.literal_eval(s) for s in ES.sec_ipc])))
ES_sec_ipc = list(pd.Series([ast.literal_eval(i) for i in ES.sec_ipc]))

toEval lenght:  9234
KS lenght:  431267


In [3]:
# Créer liste des combinaisons présentes dans ES
paires_ES = set()
for ipc_list in ES_sec_ipc:
    # Generate all pairs of IPCs in the current patent
    for pair in combinations(ipc_list, 2):
        paires_ES.add(tuple(sorted(pair)))  # Add sorted tuple to keep order consistent

print(type(paires_ES))
print(list(paires_ES)[:5])
print("Number of unique combinations: ", len(paires_ES))

<class 'set'>
[('A61M3500', 'A61P2516'), ('H04H2016', 'H04L1218'), ('B32B3712', 'B32B712'), ('F16F1502', 'F16M1122'), ('A61L210', 'F21V1306')]
Number of unique combinations:  230005


In [ ]:
def surprise_score_norm(toEval_ipc, paires_ES, thr=0):
    prop = []
    bin = []
    
    for patent in tqdm(toEval_ipc):
        patent_size = len(patent)
        if patent_size < 2:
            prop.append(0)
            bin.append(0)
            continue
        
        # Count the surprising pairs
        surprise_count = sum(1 for pair in combinations(patent, 2) if tuple(sorted(pair)) not in paires_ES)
        
        # Normalize by the total number of pairs
        total_pairs = patent_size * (patent_size - 1) / 2
        normalized_novelty = surprise_count / total_pairs
        
        prop.append(normalized_novelty)
        bin.append(1 if normalized_novelty > thr else 0)
        
    return prop, bin


def surprise_score_bin(toEval_ipc, paires_ES):
    score = []
    newPair = [[] for i in range(0, len(toEval_ipc))]

    for idx, patent in enumerate(tqdm(toEval_ipc)):
        surprise = 0
        for pair in combinations(patent, 2):
            sorted_pair = tuple(sorted(pair))
            if sorted_pair not in paires_ES:
                surprise = 1
                newPair[idx].append(sorted_pair)
        score.append(surprise)
    return(score, newPair)


In [573]:
surprise_norm = surprise_score_norm(toEval_ipc, paires_ES)
surprise_bin = surprise_score_bin(toEval_ipc, paires_ES)

100%|██████████| 9234/9234 [00:00<00:00, 210387.33it/s]


In [574]:
df = pd.DataFrame({
    'Surprise Norm': [round(i,2) for i in surprise_norm[0]],  # Renaming the first column
    'Surprise Bin': surprise_bin[0],   # Renaming the second column
    'New pairs of IPCs': surprise_bin[1]
})
# print("All occurences > 0 are the same: ", all(df[df["Surprise Norm"]>0]==df[df["Surprise Bin"]>0]))

pd.set_option('display.max_colwidth', None)
pd.concat([df, pd.DataFrame(pd.Series(toEval_ipc))], axis=1)[df["Surprise Norm"]>0]


,Surprise Norm,Surprise Bin,New pairs of IPCs,0
4,0.30,1,"[(G06F3042, G06T700), (G06F3042, G06T520), (G06F3042, G06T500)]","[G06F3041, G06T700, G06T520, G06F3042, G06T500]"
5,0.14,1,"[(G02F113, G06F303), (G02B2700, G06F303), (G02B530, G06F303)]","[G06F301, G02F113, G06T700, G02B2700, G06F303, G02B530, G02B2701]"
9,0.17,1,"[(G06F128, H02J702)]","[G06F128, H04W5202, H02J702, G06F126]"
11,1.00,1,"[(G06F120897, G06F1316)]","[G06F120897, G06F1316]"
12,0.67,1,"[(A63F1326, G06F314), (A63F1324, G06F314), (G06F30338, G06F314), (A63F1326, G06F116), (A63F1326, G06F302), (A63F1326, G06F30338), (A63F1324, G06F116), (G06F116, G06F30338), (A63F1324, G06F302), (G06F302, G06F30338)]","[G06F314, A63F1326, G06F116, A63F1324, G06F302, G06F30338]"
...,...,...,...,...
9221,0.20,1,"[(G06F708, H04L2908), (G06F708, G06T1120), (G06F708, G06T1160)]","[G06F30481, H04L2908, G06F708, G06F30484, G06T1120, G06T1160]"
9226,1.00,1,"[(G01R1925, G06F1750)]","[G06F1750, G01R1925]"
9230,0.33,1,"[(G06F950, G06Q2014), (G06F948, G06Q2014)]","[G06F950, G06F948, G06Q2014, H04L12911]"
9231,0.33,1,"[(G06F1724, G06K946)]","[G06F1724, G06K946, G06F1727]"


In [575]:
print(Counter(np.array(surprise_bin[0])))

# Statistiques
stats_surp = pd.DataFrame({
    '% surprise binary': round(len([x for x in surprise_bin[0] if x>0])/len(surprise_bin[0])*100, 2), 
}, index=[0])
stats_surp

Counter({0: 6504, 1: 2730})


,% surprise binary
0,29.56


### Distance de Jaccard des combis
- Chaque brevet est composé de paires d'IPC
- Trouver seuil de la même manière que diff, et créer variable bin ou ratio!

À faire...

### PMI - Divergences

In [20]:
def pmi(list_ipc):
    # Count occurrences of each IPC code
    ipc_counts = Counter(ipc for sublist in list_ipc for ipc in sublist)
    
    # Count co-occurrences of each IPC pair
    pair_counts = Counter()
    for sublist in list_ipc:
        for pair in combinations(sublist, 2):  # Generate all pairs in the sublist
            sorted_pair = tuple(sorted(pair))
            pair_counts[sorted_pair] += 1
    
    # Total patents considered
    total_patents = len(list_ipc)
    
    # Calculate PMI for each pair
    pmi_values_dict = {}
    for (ipc1, ipc2), pair_count in pair_counts.items():
        p_x = ipc_counts[ipc1] / total_patents
        p_y = ipc_counts[ipc2] / total_patents
        p_xy = pair_count / total_patents
        
        # Calculate PMI if none of the probabilities are zero
        if p_x > 0 and p_y > 0 and p_xy > 0:
            pmi_values_dict[(ipc1, ipc2)] = math.log(p_xy / (p_x * p_y))
    
    return pmi_values_dict

In [21]:
pmi_ES_dict = pmi(list_ipc=ES_sec_ipc)
pmi_tE_dict = pmi(list_ipc=toEval_ipc)

In [ ]:
# pmi_ES_dict2 = pmi2(list_ipc=ES_sec_ipc)
# pmi_tE_dict2 = pmi2(list_ipc=toEval_ipc)

In [ ]:
# pmi_ES_dict==pmi_ES_dict2

True

In [6]:
pmi_val_list = list(pmi_ES_dict.values())
print("Mean: ", np.mean(pmi_val_list))
print("Min: ", np.min(pmi_val_list))
print("Max: ", np.max(pmi_val_list))
print("Median: ", np.median (pmi_val_list))
print("St. deviation: ", np.std(pmi_val_list))

Mean:  5.0751831145250845
Min:  -5.96352832387434
Max:  12.974482666840602
Median:  4.9937748459709335
St. deviation:  3.139057753536195


In [700]:
print("Mean: ", np.mean(list(pmi_tE_dict.values())))

Mean:  3.4759031523833595


In [701]:
# with self ipc
import math
from collections import Counter

# Separate patents into single-IPC and multi-IPC
multi_ipc_patents = [sublist for sublist in ES_sec_ipc if len(sublist) > 1]
single_ipc_patents = [sublist[0] for sublist in ES_sec_ipc if len(sublist) == 1]
total_single_ipc_patents = len(single_ipc_patents)
total_multi_ipc_patents = len(multi_ipc_patents)

# Count occurrences of each IPC code across all patents (for pair PMI)
ipc_counts = Counter([ipc for sublist in ES_sec_ipc for ipc in sublist])

# Count co-occurrences of each IPC pair only in multi-IPC patents
pair_counts = Counter()
for sublist in multi_ipc_patents:
    for i in range(len(sublist)):
        for j in range(i + 1, len(sublist)):
            pair = tuple(sorted([sublist[i], sublist[j]]))
            pair_counts[pair] += 1

# Calculate self-PMI for single-IPC patents only
single_ipc_counts = Counter(single_ipc_patents)
self_pmi_dict = {}
for ipc, count in single_ipc_counts.items():
    p_x = count / total_single_ipc_patents
    if p_x > 0:
        self_pmi_dict[ipc] = math.log(p_x / (1 / len(single_ipc_counts)))  # Baseline of uniform probability

# Calculate PMI for IPC pairs based on multi-IPC patents
pmi_values_dict = {}
for (ipc1, ipc2), pair_count in pair_counts.items():
    p_x = ipc_counts[ipc1] / total_multi_ipc_patents
    p_y = ipc_counts[ipc2] / total_multi_ipc_patents
    p_xy = pair_count / total_multi_ipc_patents
    if p_x > 0 and p_y > 0 and p_xy > 0:
        pmi_values_dict[(ipc1, ipc2)] = math.log(p_xy / (p_x * p_y))

# Combine results into a DataFrame for analysis
import pandas as pd
pmi_df = pd.DataFrame(
    [
        {
            "IPC1": ipc1,
            "IPC2": ipc2,
            "PMI": pmi,
            "Self_PMI_IPC1": self_pmi_dict.get(ipc1, None),
            "Self_PMI_IPC2": self_pmi_dict.get(ipc2, None)
        }
        for (ipc1, ipc2), pmi in pmi_values_dict.items()
    ]
)


In [702]:
np.mean(pmi_df["PMI"])

4.276751934102647

In [703]:
list(pmi_ES_dict)
pmi_ES_dict.get(list(pmi_ES_dict.keys())[0])

3.5920675017443897

In [545]:
def new_surprise(pmi_new, pmi_known, thr_surp=0.0):
    """Compare the distribution with and without new_Q -- check for new collocations -- 
    PMI increases drastically according to a threshold.
    By setting the threshold to 0, as long as there is a new tuple we will consider it."""

    # Get the known IPC pairs from the pmi_known dictionary
    temp_known = set(pmi_known.keys())  # Using a set for faster lookups

    # Identify unique new tuples where the PMI is greater than 0 and not in known pairs
    unique_tuples = {ipc_pair for ipc_pair, pmi in pmi_new.items() 
                    if ipc_pair not in temp_known}# and pmi > 0}
    print(unique_tuples)

    counts_new = []
    # Loop through each patent in toEval_ipc
    for patent in toEval_ipc:
        count_new = 0  # Initialize new pair count for the current patent

        # Check all pairs of IPC codes in the current patent
        for i in range(len(patent)):
            for j in range(i + 1, len(patent)):
                ipc_pair = tuple(sorted([patent[i], patent[j]]))  # Sort to ensure pairs are consistent
                if ipc_pair in unique_tuples:
                    count_new += 1  # Count if the pair is in unique_tuples

        # Calculate surprise rate for the current patent
        if len(patent) > 1:  # We need at least 2 IPC codes to form a pair
            surprise_rate = count_new / (len(patent) * (len(patent) - 1) / 2)  # Total pairs possible
            counts_new.append(surprise_rate)
        else:
            counts_new.append(0)  # Append 0 if patent has no pairs

    # Determine if the surprise rate exceeds the threshold for each patent
    new_suprise = [1 if rate > thr_surp else 0 for rate in counts_new]

    return counts_new, new_suprise


In [546]:
new_surp = new_surprise(pmi_tE_dict, pmi_ES_dict)

{('G06F1206', 'G06F1340'), ('G06F1340', 'H04W1206'), ('G06Q2016', 'G09G500'), ('G06F301', 'H04N5376'), ('G06F130', 'H02M132'), ('G06F30354', 'H01H1386'), ('G05F166', 'G06F132'), ('G06F3042', 'H04N5225'), ('H04N21488', 'H04N557'), ('G06F2178', 'G06F938'), ('G06F2181', 'H04L932'), ('G06F306', 'H03M1317'), ('G11B2018', 'H04L100'), ('G06F120804', 'G06F120813'), ('G06F1320', 'G06F30488'), ('A63F1324', 'G06F30481'), ('A61B804', 'G06F1750'), ('A44C900', 'G06F316'), ('G06F2153', 'G06F2188'), ('B60R16023', 'G06F126'), ('A61L210', 'G06F30488'), ('G06F2141', 'G06T700'), ('G06F1208', 'G06F1310'), ('G06F132', 'H03M1315'), ('A47F300', 'G06F30488'), ('G06F1724', 'G06K946'), ('G06F30484', 'H04N21435'), ('G01S1953', 'G05D104'), ('G06F1202', 'G06F1316'), ('G06F1342', 'G06F30482'), ('C23C1404', 'C23C1456'), ('G11B27031', 'H04N216334'), ('G06F120', 'G06F950'), ('H04L912', 'H04N144'), ('G06F1340', 'G06F316'), ('B60K3706', 'G06F3147'), ('G06F1718', 'H04L908'), ('G06F7487', 'G06F7535'), ('G06F1130', 'G06F954

In [548]:
print(Counter(new_surp[1]))
2730/9234



Counter({0: 6504, 1: 2730})


0.29564652371669914

In [179]:
from collections import OrderedDict

# pmi_tE_dict.values()
list(OrderedDict.fromkeys(item[0][1] for item in pmi_tE_dict))

list(OrderedDict.fromkeys([item for item in pmi_tE_dict]))

[('G06F301', 'G06F3041'),
 ('G06F312', 'G06Q3004'),
 ('G06F1730', 'G06F30482'),
 ('G06F1730', 'G06F30484'),
 ('G06F30482', 'G06F30484'),
 ('G06F3041', 'G06T700'),
 ('G06F3041', 'G06T520'),
 ('G06F3041', 'G06F3042'),
 ('G06F3041', 'G06T500'),
 ('G06T520', 'G06T700'),
 ('G06F3042', 'G06T700'),
 ('G06T500', 'G06T700'),
 ('G06F3042', 'G06T520'),
 ('G06T500', 'G06T520'),
 ('G06F3042', 'G06T500'),
 ('G02F113', 'G06F301'),
 ('G06F301', 'G06T700'),
 ('G02B2700', 'G06F301'),
 ('G06F301', 'G06F303'),
 ('G02B530', 'G06F301'),
 ('G02B2701', 'G06F301'),
 ('G02F113', 'G06T700'),
 ('G02B2700', 'G02F113'),
 ('G02F113', 'G06F303'),
 ('G02B530', 'G02F113'),
 ('G02B2701', 'G02F113'),
 ('G02B2700', 'G06T700'),
 ('G06F303', 'G06T700'),
 ('G02B530', 'G06T700'),
 ('G02B2701', 'G06T700'),
 ('G02B2700', 'G06F303'),
 ('G02B2700', 'G02B530'),
 ('G02B2700', 'G02B2701'),
 ('G02B530', 'G06F303'),
 ('G02B2701', 'G06F303'),
 ('G02B2701', 'G02B530'),
 ('G06F2156', 'G06F2157'),
 ('G06F2156', 'G06F2162'),
 ('G06F2157', 

In [ ]:
# flat_list = [item for sublist in toEval_ipc for item in sublist]
# """ Input : list of ORDERED feature variables (words for example) -- if feature order does not matter set window_size to inf. """
# bigram_measures = BigramAssocMeasures()
# finder = BigramCollocationFinder.from_words(flat_list, window_size= 2)
# temp_pmi = finder.score_ngrams(bigram_measures.pmi)

In [ ]:
# print(temp_pmi[:5])
# print(len(temp_pmi))

[(('A23F318', 'A23F526'), 14.46326891598129), (('A23F526', 'A47J3152'), 14.46326891598129), (('A44C900', 'A44C1500'), 14.46326891598129), (('A47F300', 'A47F1106'), 14.46326891598129), (('A47G106', 'A47G114'), 14.46326891598129)]
10740


In [ ]:
# sorted(temp_pmi)[:5]

[(('A01G100', 'G06F3041'), 4.705045701254566),
 (('A23F318', 'A23F526'), 14.46326891598129),
 (('A23F526', 'A47J3152'), 14.46326891598129),
 (('A41B108', 'A41D100'), 13.46326891598129),
 (('A41D100', 'A61B500'), 8.46326891598129)]

In [ ]:
# list(OrderedDict.fromkeys(item[0][1] for item in temp_pmi))[:5]

['A23F526', 'A47J3152', 'A44C1500', 'A47F1106', 'A47G114']

In [383]:
# Convert to list of tuples
temp_pmi = [((ipc1, ipc2), value) for (ipc1, ipc2), value in pmi_tE_dict.items()]

# Output
sorted(temp_pmi)[:5]

[(('A01G100', 'G06F1724'), 3.5020163500887067),
 (('A23F318', 'A23F526'), 8.759511722116487),
 (('A23F318', 'A47J3152'), 8.759511722116487),
 (('A23F318', 'G06F316'), 3.431635553326907),
 (('A23F526', 'A47J3152'), 8.759511722116487)]

In [459]:
column_names0 = list(OrderedDict.fromkeys(item[0][0] for item in temp_pmi))
column_names1 = list(OrderedDict.fromkeys(item[0][1] for item in temp_pmi))

In [476]:
print(sorted(column_names0)==sorted(column_names1))
print(len(column_names0))
print(len(column_names1))
all(item in column_names0 for item in column_names1)

False
1283
1330


False

In [384]:
""" Take a PMI list of tuples in nltk format [((w1,w2),value)] and output a nested dictionary """
nested_dict = {}
column_names = list(OrderedDict.fromkeys(item[0][1] for item in temp_pmi))

for (key1, key2), value in temp_pmi:
    if key1 not in nested_dict:
        nested_dict[key1] = {}
    nested_dict[key1][key2] = value
nested_dict['variables'] = column_names

In [387]:
def convert_dict(dict):    
    original_dict = dict
    converted_dict = {}
    # column_names = [item[i][0][1] for i in list(dict.keys()) for item in list(dict.keys())]
    column_names = list(OrderedDict.fromkeys(item[1] for item in list(dict.keys())))
    # print(column_names)
    for (key1, key2), value in original_dict.items():
        if key1 not in converted_dict:
            converted_dict[key1] = {}
        converted_dict[key1][key2] = value
    converted_dict['variables'] = column_names
    return converted_dict


tE_dict_conv = convert_dict(pmi_tE_dict)
ES_dict_conv = convert_dict(pmi_ES_dict)
# Output the converted dictionary
# print(tE_dict_conv)
# pprint(ES_dict_conv)

# print(pmi_tE_dict)
# (pmi_ES_dict)
# print(column_names)

In [403]:
print(len(nested_dict))
print(len(tE_dict_conv))
print(nested_dict==tE_dict_conv)

1284
1284
True


In [477]:
epsilon = 0.01
variables_1 = ES_dict_conv['variables']
variables_2 = tE_dict_conv['variables']
inter_list = list(set(variables_1 + variables_2))

print(len(variables_1))
print(len(variables_2))
print(len(inter_list))


14693
1330
14799


In [ ]:
print(tE_dict_conv.get('G01L906'))

None


'G01L906'

In [479]:
# get common vectors

epsilon = 0.01
variables_1 = ES_dict_conv['variables']
variables_2 = tE_dict_conv['variables']
inter_list = list(set(variables_1 + variables_2))

# print(variables_1)

vectors = {}
for entry in tqdm(inter_list):
    vec_1 = [ES_dict_conv.get(entry, {}).get(key, epsilon) for key in list(ES_dict_conv.keys())]
    vec_2 = [tE_dict_conv.get(entry, {}).get(key, epsilon) for key in list(tE_dict_conv.keys())]
    vectors[entry] = (vec_1, vec_2)

100%|██████████| 14799/14799 [05:23<00:00, 45.77it/s]


In [489]:
print([i for i, val in enumerate(inter_list) if val == 'G01L906'])
print([(idx, item) for (idx, item) in enumerate(vectors['G01L906'][0]) if item!=0.01])
print([(idx, item) for (idx, item) in enumerate(vectors['G01L906'][1]) if item!=0.01])

[0]
[(904, 5.229075494282744)]
[]


In [490]:
print(inter_list[0])
print(list(ES_dict_conv.keys())[904])
ES_dict_conv["G01L906"]
# tE_dict_conv["G01L906"]

G01L906
H05K1304


{'H05K1304': 5.229075494282744}

In [494]:
vecotr_tuples = vectors
key_list = vecotr_tuples.keys()
surprise_dists = []

len(key_list)==len(inter_list)

True

In [500]:
from ipcMetrics.divergences import Distance

for entry in key_list:
    tuple_known = vecotr_tuples[entry][0]
    tuple_new = vecotr_tuples[entry][1]
    
    #### We want only positive PMI score -- no negative values for not going nan or inf values
    tuple_known = [max(0., val) for val in tuple_known]
    tuple_new = [max(0., val) for val in tuple_new]   
    if sum(tuple_known) != 0 and sum(tuple_new) != 0:   #### We can't compare to non existing vectors neither - utilité???? car on popule de epsilon!
        surprise_dists.append(Distance.jacc_matrix(tuple_known, tuple_new))


AttributeError: type object 'Distance' has no attribute 'jacc_matrix'

In [ ]:
surprise_score = sum(surprise_dists) / len(surprise_dists)
dist_surprise = 0
if surprise_score > thr_surp:
    dist_surprise = 1
            

In [33]:
pmi_surprise = []

for patent in tqdm(toEval_ipc):
    # Generate pairs from the new patent
    new_patent_pairs = []
    for i in range(len(patent)):
        for j in range(i + 1, len(patent)):
            pair = tuple(sorted([patent[i], patent[j]]))
            new_patent_pairs.append(pair)
    # Sum the PMI values of the pairs in the new patent
    pmi_sum = 0
    for pair in new_patent_pairs:
        pmi = pmi_ES_dict.get(pair, None)
        if pmi is None:
            pmi = np.max(list(pmi_ES_dict.values()))  # Arbitraire
        pmi_sum += pmi
    pmi_surprise.append(pmi_sum)
    

  7%|▋         | 647/9234 [00:09<02:01, 70.89it/s] 


KeyboardInterrupt: 

In [34]:
from collections import defaultdict

epsilon = np.max(list(pmi_ES_dict.values()))
pmi_ES_dict_lambda = defaultdict(lambda: epsilon, pmi_ES_dict)

pmi_surprise = []

for patent in toEval_ipc:
    # Sum the PMI values of the pairs in the new patent
    pmi_sum = sum(
        pmi_ES_dict_lambda[tuple(sorted([patent[i], patent[j]]))]
        for i in range(len(patent))
        for j in range(i + 1, len(patent))
    )
    pmi_surprise.append(pmi_sum)

In [11]:
pmi_surprise = []
bin=[]

for patent in tqdm(toEval_ipc):
    # Sum the PMI values of the pairs in the new patent
    pmi_sum = sum(
        pmi_ES_dict_lambda[tuple(sorted([patent[i], patent[j]]))]
        for i in range(len(patent))
        for j in range(i + 1, len(patent))
    )
    pmi_surprise.append(pmi_sum)
    bin.append(1 if pmi_sum > 0 else 0)

  0%|          | 0/9234 [00:00<?, ?it/s]

100%|██████████| 9234/9234 [00:00<00:00, 62897.40it/s]


In [19]:
len(pmi_ES_dict)
len(pmi_ES_dict_lambda)
epsilon

12.974482666840602

In [27]:
print(np.mean(pmi_surprise))
print(np.mean(bin))

94.17637767292642
0.6846437080355209


In [677]:
np.corrcoef(surprise_bin[0], surprise_norm[0])

array([[1.        , 0.84737819],
       [0.84737819, 1.        ]])

In [678]:
np.corrcoef(pmi_surprise, surprise_norm[0])

array([[1.        , 0.36115185],
       [0.36115185, 1.        ]])

In [679]:
np.corrcoef([int(b) for b in list(pd.Series(pmi_surprise)<2.5)], surprise_bin[0])

array([[ 1.        , -0.50988277],
       [-0.50988277,  1.        ]])

In [40]:
import importlib
from ipcMetrics import divergences
importlib.reload(divergences)

<module 'ipcMetrics.divergences' from 'c:\\Users\\edgar\\OneDrive\\Bureau\\Ecole\\HEC\\A24\\BrevetNLP\\PatentNovelty\\ipcMetrics\\divergences.py'>

In [85]:
divergences.Distance.pmi(ES_sec_ipc[10], w_size=len(ES_sec_ipc[10]))

[(('H05K500', 'H01H1314'), 0.5849625007211562),
 (('H05K506', 'H01H1314'), 0.5849625007211562),
 (('H05K506', 'H05K500'), 0.5849625007211562)]

In [90]:
pmi_val_list = list(pmi_values.values())
print("Mean: ", np.mean(pmi_val_list))
print("Min: ", np.min(pmi_val_list))
print("Max: ", np.max(pmi_val_list))
print("Median: ", np.median (pmi_val_list))
print("St. deviation: ", np.std(pmi_val_list))

AttributeError: 'list' object has no attribute 'values'